# Downloading the dlib

**Link to download shape_predictor_68_face_landmarks.dat**
https://github.com/italojs/facial-landmarks-recognition/blob/master/shape_predictor_68_face_landmarks.dat

**Link to download dlib whl for Python Version 3.11**
https://github.com/Murtaza-Saeed/dlib/blob/master/dlib-19.24.1-cp311-cp311-win_amd64.whl

# Detection Code

In [2]:
import cv2
import dlib
from scipy.spatial import distance as dist

***Function to calculate the eye aspect ratio (EAR)***

In [3]:
def eye_aspect_ratio(eye):
    # Euclidean distances between the vertical eye landmarks
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    # Euclidean distance between the horizontal eye landmarks
    C = dist.euclidean(eye[0], eye[3])

    # eye aspect ratio
    ear = (A + B) / (2.0 * C)
    return ear

In [4]:
predictor_path = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_path)

*Defining Constants for eye landmarks & Constants for drowsiness detection*

In [5]:
LEFT_EYE_POINTS = list(range(36, 42))
RIGHT_EYE_POINTS = list(range(42, 48))

EAR_THRESHOLD = 0.25 
CONSEC_FRAMES = 20

In [6]:
frame_counter = 0
drowsy_frames = 0

# Open the webcam
cap = cv2.VideoCapture(0)

In [7]:
while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale frame
    faces = detector(gray)

    for face in faces:
        # Get facial landmarks
        shape = predictor(gray, face)
        shape = [(shape.part(i).x, shape.part(i).y) for i in range(68)]

        # Extract eye landmarks
        left_eye = shape[36:42]  
        right_eye = shape[42:48] 

        # Calculate eye aspect ratio for each eye
        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)

        # Average eye aspect ratio
        avg_ear = (left_ear + right_ear) / 2.0

        # Draw the facial landmarks on the frame
        for (x, y) in shape:
            cv2.circle(frame, (x, y), 1, (0, 0, 255), -1)

        # Drowsiness detection
        if avg_ear < EAR_THRESHOLD:
            drowsy_frames += 1
            if drowsy_frames >= CONSEC_FRAMES:
                cv2.putText(frame, "Drowsiness Detected", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        else:
            drowsy_frames = 0

        # Display the eye aspect ratio on the frame
        cv2.putText(frame, f"EAR: {avg_ear:.2f}", (300, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    # Display the frame
    cv2.imshow("Drowsiness Detection", frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()